In [49]:
import pandas as pd
import numpy as np
df = pd.read_csv("airlines.csv")

# delete 10% of specified column
df.loc[df.sample(frac=0.1).index, "time"] = np.nan
# df.info()
df.isnull().sum()

flight             0
time           53938
length             0
airline            0
airportfrom        0
airportto          0
dayofweek          0
delay              0
dtype: int64

In [50]:
# Drop flight column since it has identifier that shouldn't affect other columns
df = df.drop(['flight'], axis=1)

In [51]:
df

,time,length,airline,airportfrom,airportto,dayofweek,delay
0,1235.0,80.0,MQ,DFW,CRP,5,0
1,1296.0,141.0,DL,ATL,HOU,1,0
2,360.0,146.0,OO,COS,ORD,4,0
3,1170.0,143.0,B6,BOS,CLT,3,0
4,NaN,344.0,US,OGG,PHX,6,0
...,...,...,...,...,...,...,...
539378,530.0,72.0,OO,GEG,SEA,5,1
539379,560.0,115.0,WN,LAS,DEN,4,1
539380,827.0,74.0,EV,CAE,ATL,2,1
539381,715.0,65.0,WN,BWI,BUF,4,1


In [52]:
# one hot encode categorical values
cat_variables = df[['airline', 'airportfrom', 'airportto']]
cat_dummies = pd.get_dummies(cat_variables, drop_first=True)
# cat_dummies.head()
df = df.drop(['airline', 'airportfrom', 'airportto'], axis=1)
df = pd.concat([df, cat_dummies], axis=1)
df.head()

,time,length,dayofweek,delay,airline_AA,airline_AS,airline_B6,airline_CO,airline_DL,airline_EV,...,airportto_TXK,airportto_TYR,airportto_TYS,airportto_UTM,airportto_VLD,airportto_VPS,airportto_WRG,airportto_XNA,airportto_YAK,airportto_YUM
0,1235.0,80.0,5,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1296.0,141.0,1,0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,360.0,146.0,4,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1170.0,143.0,3,0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,NaN,344.0,6,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
# Separate the null value rows from dataframe (df) and create a variable “test data”
test_data = df[df['time'].isnull()]

# Drop the null values from the dataframe (df) and represent as ‘train data”
df.dropna(inplace=True)

In [54]:
# Use all other columns to predict time
x_train = df.drop(['time'], axis=1)
x_train

,length,dayofweek,delay,airline_AA,airline_AS,airline_B6,airline_CO,airline_DL,airline_EV,airline_F9,...,airportto_TXK,airportto_TYR,airportto_TYS,airportto_UTM,airportto_VLD,airportto_VPS,airportto_WRG,airportto_XNA,airportto_YAK,airportto_YUM
0,80.0,5,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,141.0,1,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,146.0,4,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,143.0,3,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,98.0,4,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539378,72.0,5,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539379,115.0,4,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539380,74.0,2,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
539381,65.0,4,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [55]:
# target variable is time
y_train = df['time']
y_train

0         1235.0
1         1296.0
2          360.0
3         1170.0
5          692.0
           ...  
539378     530.0
539379     560.0
539380     827.0
539381     715.0
539382     770.0
Name: time, Length: 485445, dtype: float64

In [56]:
# Build the linear regression model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [57]:
x_test = test_data.drop(['time'], axis=1)
y_test = test_data['time']
y_test

4        NaN
10       NaN
13       NaN
19       NaN
41       NaN
          ..
539332   NaN
539348   NaN
539349   NaN
539367   NaN
539370   NaN
Name: time, Length: 53938, dtype: float64

In [58]:
# Apply the model on x_test of test data to make predictions.
y_pred = lr.predict(x_test)

In [64]:
# Get the correct answers from original dataset
original = pd.read_csv("airlines.csv")
original = original.loc[y_test.index, "time"]
original

4         1410.0
10         530.0
13        1215.0
19        1160.0
41        1071.0
           ...  
539332     572.0
539348     900.0
539349    1345.0
539367    1149.0
539370     865.0
Name: time, Length: 53938, dtype: float64

In [67]:
from sklearn.metrics import mean_squared_error
# root mean squared error
rms = np.sqrt(mean_squared_error(original, y_pred))
rms

266.6136672528233